In [1]:
# import sys

import numpy as np
from warnings import warn

from aif360.datasets import GermanDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from common_utils import compute_metrics
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        import load_preproc_data_german
from sklearn.preprocessing import MaxAbsScaler
from aif360.algorithms.inprocessing import MetaFairClassifier
from aif360.algorithms.postprocessing import RejectOptionClassification


from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display

2024-04-30 09:40:54.055738: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/jessicachen/opt/anaconda3/envs/226/lib/python3.8/site-packages/torch/_functorch/deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')


In [2]:
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]
dataset_orig = load_preproc_data_german(['age'])

In [3]:
# split data into epochs, each with a different group of agents
NUM_EPOCHS = 2
dataset_orig_epochs = dataset_orig.split(NUM_EPOCHS, shuffle=True)
print("Size of epoch: ", dataset_orig_epochs[0].features.shape)

Size of epoch:  (500, 11)


In [11]:
# NO FAIRNESS
# takes two epochs of data
# trains on epoch_1, then uses model to classify epoch 2

def no_fairness_train_and_classify(data_epoch_1, data_epoch_2):
    # print out some labels, names, etc.
    display(Markdown("#### No Fairness"))
    print("Epoch 1: ", data_epoch_1.features.shape)
    print("Epoch 2: ",data_epoch_2.features.shape)

    # train classifier on epoch 1
    scale_orig = StandardScaler()
    X_train = scale_orig.fit_transform(data_epoch_1.features)
    y_train = data_epoch_1.labels.ravel()
    lmod = LogisticRegression(solver='liblinear')  # Solver specified to avoid future warnings
    lmod.fit(X_train, y_train)

    # classify epoch 2 agents
    X_epoch2 = scale_orig.fit_transform(data_epoch_2.features)
    y_epoch2_pred = lmod.predict(X_epoch2)
    data_epoch_2_pred = data_epoch_2.copy(deepcopy=True)
    data_epoch_2_pred.labels = y_epoch2_pred

    # Evaluate fairness metrics on classification of epoch 2
    metric_train = BinaryLabelDatasetMetric(data_epoch_2_pred, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
    
    print("Training set: Difference in mean outcomes = {:.3f}".format(metric_train.mean_difference()))

    metric_test_aft = compute_metrics(data_epoch_2, data_epoch_2_pred, 
                unprivileged_groups, privileged_groups)


In [14]:
def ROC_train_and_classify(data_epoch_1, data_epoch_2):
    # print out some labels, names, etc.
    display(Markdown("#### ROC Fairness"))
    print("Epoch 1: ", data_epoch_1.features.shape)
    print("Epoch 2: ",data_epoch_2.features.shape)

    # Metric used (should be one of allowed_metrics)
    metric_name = "Statistical parity difference"

    # Upper and lower bound on the fairness metric used
    metric_ub = 0.05
    metric_lb = -0.05

    scale_orig = StandardScaler()

    # need to first train a model to get predicted scores
    X_train = scale_orig.fit_transform(data_epoch_1.features)
    y_train = data_epoch_1.labels.ravel()
    lmod = LogisticRegression(solver='liblinear')  # Solver specified to avoid future warnings
    lmod.fit(X_train, y_train)

    # indices of favorable label
    pos_ind = np.where(lmod.classes_ == data_epoch_1.favorable_label)[0][0]

    # data_epoch_1_pred contains PREDICTED SCORES
    # use same epoch 1 data instead of separate validation
    data_epoch_1_pred = data_epoch_1.copy(deepcopy=True)
    X_train = scale_orig.transform(data_epoch_1_pred.features)
    data_epoch_1_pred.scores = lmod.predict_proba(X_train)[:,pos_ind].reshape(-1,1)

    ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                 privileged_groups=privileged_groups, 
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                  num_class_thresh=100, num_ROC_margin=50,
                                  metric_name=metric_name,
                                  metric_ub=metric_ub, metric_lb=metric_lb)
    ROC = ROC.fit(data_epoch_1, data_epoch_1_pred)

    print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
    print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

    # Metrics for the transformed test set
    data_epoch_2_pred = ROC.predict(data_epoch_2)

    # Evaluate fairness metrics on classification of epoch 2
    metric_train = BinaryLabelDatasetMetric(data_epoch_2_pred, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
    
    print("Training set: Difference in mean outcomes = {:.3f}".format(metric_train.mean_difference()))

    metric_test_aft = compute_metrics(data_epoch_2, data_epoch_2_pred, 
                unprivileged_groups, privileged_groups)

In [15]:
no_fairness_train_and_classify(dataset_orig_epochs[0],dataset_orig_epochs[1])
ROC_train_and_classify(dataset_orig_epochs[0],dataset_orig_epochs[1])

#### No Fairness

Epoch 1:  (500, 11)
Epoch 2:  (500, 11)
Training set: Difference in mean outcomes = -0.644
Balanced accuracy = 0.5273
Statistical parity difference = -0.6444
Disparate impact = 0.3556
Average odds difference = -0.6750
Equal opportunity difference = -0.5833
Theil index = 0.1283


#### ROC Fairness

Epoch 1:  (500, 11)
Epoch 2:  (500, 11)
Optimal classification threshold (with fairness constraints) = 0.6237
Optimal ROC margin = 0.1613
Training set: Difference in mean outcomes = -0.043
Balanced accuracy = 1.0000
Statistical parity difference = -0.0431
Disparate impact = 0.9393
Average odds difference = 0.0000
Equal opportunity difference = 0.0000
Theil index = 0.0000
